## **Simulando uma API com Flask para perguntas e respostas**

In [1]:
# Instalando o Flask (para criar a API) e flask-cors (para permitir requisições de fora)
!pip install flask flask-cors

## **Crie a API com Flask no próprio notebook**

In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import pipeline  # Pipeline de perguntas e respostas do Hugging Face
from pathlib import Path  # Ler o arquivo de texto
import threading  # Rodar a API em background dentro do Jupyter

# Inicializa o Flask
app = Flask(__name__)
CORS(app)  # Permite chamadas de outros domínios

# Carrega o modelo de perguntas e respostas
qa_model = pipeline(
    "question-answering",
    model="pierreguillou/bert-base-cased-squad-v1.1-portuguese",
    tokenizer="pierreguillou/bert-base-cased-squad-v1.1-portuguese"
)

# Carrega o contexto
contexto = Path("/content/leitor_nlp/data/textp_limpo.txt").read_text(encoding="utf-8")

# Define a rota POST /perguntar
@app.route("/perguntar", methods=["POST"])
def responder():

  # Lê o JSON da requisição (espera: {"pergunta": "sua pergunta"})
  dados = request.json
  pergunta = dados.get("pergunta")

  # Verifica se a pergunta foi enviada
  if not pergunta:
    return jsonify({"erro": "Pergunta não fornecida"}), 400

  # Usa o modelo para responder com base no contexto
  resposta = qa_model(question=pergunta, context=contexto)

  # Retorna a resposta em JSON
  return jsonify({
      "pergunta": pergunta,
      "resposta": resposta["answer"],
      "score": round(resposta["score"], 2)
  })

# Roda o Flask em um thread (para funcionar no Jupyter)
def iniciar_api():
  app.run(port=5000)

# Inicia a API em uma thread separada
thread = threading.Thread(target=iniciar_api)
thread.start()

config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


 * Serving Flask app '__main__'


## **Teste a API com uma requisição local no notebook**

In [4]:
# Faz uma requisição POST para a API com uma pergunta
import requests

# URL da API local
url = "http://127.0.0.1:5000/perguntar"

# Exemplo de pergunta
pergunta = {"pergunta": "Qual o valor do contrato?"}

# Envia a requisição
resposta = requests.post(url, json=pergunta)

# Mostra os resultados
print("🔎 Pergunta:", pergunta["pergunta"])
print("✅ Resposta da API:", resposta.json())

INFO:werkzeug:127.0.0.1 - - [15/May/2025 01:19:49] "POST /perguntar HTTP/1.1" 200 -


🔎 Pergunta: Qual o valor do contrato?
✅ Resposta da API: {'pergunta': 'Qual o valor do contrato?', 'resposta': 'r 12.000,00 doze mil reais', 'score': 0.43}
